In [1]:
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image

In [2]:
from network.mynetwork_uu_bg import Unet as Uunet
from network.mynetwork import Unet
from network.mynetwork_cmp import Unet as Unet_cmp
from network.styler import Unet as Unet_styler
from loss.loss import CLIPLoss


In [3]:
from thop import profile
from torchsummary import summary


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

input_pic = torch.ones(1, 3, 512, 512).to(device)
input1 = torch.ones(1, 3, 244, 244).to(device).long()
input2 = torch.ones(1, 77).to(device).long()

In [5]:
print("clip")
model = CLIPLoss(device).to(device)
model_clip = model.model
clip_flops, clip_params = profile(model_clip, inputs=(input1, input2), verbose=False)
print('FLOPs = ' + str(clip_flops/(1000**3)) + 'G')
print('Params = ' + str(clip_params/(1000**2)) + 'M')

clip
FLOPs = 4.884529152G
Params = 84.225024M


In [6]:
print("vgg19")
model_vgg = torchvision.models.vgg19(pretrained=True).features.to(device)
vgg_flops, vgg_params = profile(model_vgg, inputs=(input_pic,), verbose=False)
print('FLOPs = ' + str(vgg_flops/(1000**3)) + 'G')
print('Params = ' + str(vgg_params/(1000**2)) + 'M')

vgg19
FLOPs = 101.9215872G
Params = 20.024384M


In [7]:
print("mine")
model_mine = Uunet(device).to(device)
mine_flops, mine_params = profile(model_mine, inputs=(input_pic,), verbose=False)
print('FLOPs = ' + str((mine_flops)/(1000**3)) + 'G')
print('Params = ' + str((mine_params)/(1000**2)) + 'M')



mine
FLOPs = 60.609789952G
Params = 2.562883M


In [8]:
print("mine_u")
model_mine_u = Unet(device).to(device)
mine_u_flops, mine_u_params = profile(model_mine_u, inputs=(input_pic,), verbose=False)
print('FLOPs = ' + str((mine_u_flops+clip_flops)/(1000**3)) + 'G')
print('Params = ' + str((mine_u_params+clip_params)/(1000**2)) + 'M')

mine_u
FLOPs = 58.46990848G
Params = 86.858755M


In [9]:
# print("cmp")
# model = Unet_cmp(device).to(device)
# cmp_flops, cmp_params = profile(model, inputs=(input_pic,), verbose=False)
# print('FLOPs = ' + str((cmp_flops+clip_flops+vgg_flops)/(1000**3)) + 'G')
# print('Params = ' + str((cmp_params+clip_params+vgg_params)/(1000**2)) + 'M')

In [10]:
print("styler")
model = Unet_styler().to(device)
sty_flops, sty_params = profile(model, inputs=(input_pic,), verbose=False)
print('FLOPs = ' + str((sty_flops)/(1000**3)) + 'G')
print('Params = ' + str((sty_params)/(1000**2)) + 'M')



styler
FLOPs = 16.252928G
Params = 0.615267M


In [11]:
# print("vgg")
# summary(model_vgg,input_size=input_pic.shape[1:])

In [12]:
# print("mine")
# summary(model_mine,input_size=input_pic.shape[1:])

In [13]:
# print("mine_u")
# summary(model_mine_u,input_size=input_pic.shape[1:])


In [14]:
# print("styler")
# summary(model,input_size=input_pic.shape[1:])

